In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

from SpeechIO import stt,tts
from FaceSave import Capture



from PIL import ImageGrab

In [ ]:


Capture()
path = 'Faces/Captured'
images = []
classNames = [] #names of captured images without extension
myList = os.listdir(path)
print(myList)
n=-1
for cl in myList:
    n+=1
    curImg = cv2.imread(f'{path}/{cl}')
    if curImg.all:
        images.append(curImg)
        classNames.append(os.path.splitext(cl)[0])
        print(classNames[n])
    else:
        print(f'Couldnt read {cl}')


def findEncodings(images):
    encodeList = []
    n=-1 #counter used to find name of image if needed
    for img in images:
        n+=1
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        try:
            encode = face_recognition.face_encodings(img)[0]
            
        except IndexError:
            print(f"Couldnt detect {classNames[n]}'s face, Skipping...")
            continue
        if encode.any:
            print(f'encoding {myList[n]}')
            encodeList.append(encode)

    if encodeList:
        return encodeList
    else:
        print('couldnt find a single face, exiting....')
        exit()
 
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
    for line in myDataList:
        entry = line.split(',')
        nameList.append(entry[0])
    if name not in nameList:
        now = datetime.now()
        dtString = now.strftime('%H:%M:%S')
        f.writelines(f'n{name},{dtString}')
 
#### FOR CAPTURING SCREEN RATHER THAN WEBCAM
# def captureScreen(bbox=(300,300,690+300,530+300)):
#     capScr = np.array(ImageGrab.grab(bbox))
#     capScr = cv2.cvtColor(capScr, cv2.COLOR_RGB2BGR)
#     return capScr
 
encodeListKnown = findEncodings(images)
print('Encoding Complete')


In [ ]:
cap = cv2.VideoCapture(0)
 
while True:
    success, img = cap.read()
    #img = captureScreen()
    imgS = cv2.resize(img,(0,0),None,0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    small_frame = cv2.resize(imgS, (0, 0), fx=0.1, fy=0.1)
    facesCurFrame = face_recognition.face_locations(small_frame)
    encodesCurFrame = face_recognition.face_encodings(small_frame,facesCurFrame)


    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        
        matches = face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown,encodeFace)
        #print(faceDis)
        matchIndex = np.argmin(faceDis)
    
    if matches[matchIndex]:
        name = classNames[matchIndex].upper()
        #print(name)
        y1,x2,y2,x1 = faceLoc
        y1, x2, y2, x1 = y1*4,x2*4,y2*4,x1*4
        cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
        cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
        cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
        #markAttendance(name)

    cv2.imshow('Webcam',img)
    cv2.waitKey(1)